**reference**
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=DWgnNJrYW2GP

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
file_folder = r"hi_train.conll"
file =  open(file_folder,encoding="utf-8")
text = file.read()


FileNotFoundError: ignored

In [ ]:
# reading lines and saving words and tags in separate lists
s_cout = 0
l1 = []
l2 = []
l3 = []
for word in text.split('\n'):
  if(word == ''):
    s_cout =  s_cout + 1
  elif(word[0] == '#'):
    continue
    
  else:
      word = word.split(" ")
      k  = "sent_"+ str(s_cout)
      l1.append(k)
      l2.append(word[0])
      l3.append(word[-1])



In [ ]:
# converting in pandas data frame
df = pd.DataFrame({"sent":l1,"word":l2,"tag":l3})

In [ ]:
df['sentence'] =  df['word']
df['word_labels']  = df['tag']
df.head()

,sent,word,tag,sentence,word_labels
0,sent_0,जियोर्जियोन,O,जियोर्जियोन,O
1,sent_0,०१९।,O,०१९।,O
2,sent_0,जेपीजी,O,जेपीजी,O
3,sent_0,|,O,|,O
4,sent_0,जियोर्जियोन,O,जियोर्जियोन,O


In [ ]:
# removed rows with empty word
lis = []
for i in range(df.shape[0]):
  if(df.iloc[i,1] != ''):
    lis.append(i)
df = df.iloc[lis,:]

In [ ]:
# mapping tags to numbers and vice - versa
labels_to_ids = {k: v for v, k in enumerate(df.tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(df.tag.unique())}
labels_to_ids
ids_to_labels

{0: 'O',
 1: 'B-CW',
 2: 'B-PROD',
 3: 'B-PER',
 4: 'I-PER',
 5: 'I-PROD',
 6: 'B-LOC',
 7: 'B-CORP',
 8: 'I-CORP',
 9: 'B-GRP',
 10: 'I-GRP',
 11: 'I-CW',
 12: 'I-LOC'}

In [ ]:
# removing duplicates indices from df
df = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
df.head()

,sentence,word_labels
0,जियोर्जियोन,O
1,०१९।,O
2,जेपीजी,O
3,|,O
4,आंधी,B-CW


In [ ]:
df['word_labels'].unique()

array(['O', 'B-CW', 'B-PROD', 'B-PER', 'I-PER', 'I-PROD', 'B-LOC',
       'B-CORP', 'I-CORP', 'B-GRP', 'I-GRP', 'I-CW', 'I-LOC'],
      dtype=object)

In [ ]:
# installing transformers sentencepiece
!pip install transformers
!pip install sentencepiece

In [ ]:
# importing transformers and fixing parameters values 
import transformers

MAX_LEN = 20
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 6*1e-03
MAX_GRAD_NORM = 10
from transformers import AutoModel, AutoTokenizer
# imported tokeinzer form ai4bharat/indic-bert
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')


In [ ]:
#For evaluation
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader


In [ ]:

class dataset(Dataset):
  # defining parameters
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip()
        word_labels = self.data.word_labels[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        encoded_labels = labels
      

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(labels[0])
        
        return item

  def __len__(self):
        return self.len

In [ ]:
# train size = .8
# validation size - .2
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)

temp1 = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

val_dataset = temp1

val_dataset = val_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))


training_set = dataset(train_dataset, tokenizer, MAX_LEN)
validation_set = dataset(val_dataset, tokenizer, MAX_LEN)


NameError: ignored

In [ ]:
# defining training_loader and testing loader for preparing data for training and testing, using predefined batch size 
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)

NameError: ignored

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
import torch
import torch.nn as nn

In [ ]:
# defining class
class clyas(nn.Module):
  # if freeze_bert=False = True , then bert model will not be trained
    def __init__(self,freeze_bert=False):
     super(clyas, self).__init__()
     # auto model has pretrained bert parameters
     self.auto =  AutoModel.from_pretrained('ai4bharat/indic-bert')
     self.classifier =  nn.Sequential(nn.Linear(768, 150), nn.ReLU(),nn.Linear(150, 80),nn.ReLU(),nn.Dropout(0.15),nn.Linear(80, 45),nn.ReLU(),nn.Dropout(0.15),nn.Linear(45,13))
     if freeze_bert:
            for param in self.auto.parameters():
                param.requires_grad = False
  # forward function , it gets cls state embedding for every word from pretrained bert model and pass it as a input to the classifer 
  def forward(self,ids,mask):
    output = self.auto(input_ids=ids,attention_mask=mask)
    # cls embedding for every word
    hidden_stat = output[0][:, 0, :]
    # return lof=gits value
    logits = self.classifier(hidden_stat)
    return logits




In [ ]:
# cross entropy loss function for loss calculetion
loss_fn = nn.CrossEntropyLoss()
bertt = clyas(True)
bertt.to(device)

clyas(
  (auto): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
               

In [ ]:
def train(epochs):
  
  tr_loss, tr_accuracy = 0, 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # stores predicted value and labels for loss 
  tr_preds, tr_labels = [], []
  # adam optimizer
  optimizer = torch.optim.Adam(params=bertt.parameters(), lr=LEARNING_RATE)
  bertt.train()
  # training loader
  for idx, batch in enumerate(training_loader):
    # ids, mask and label assigned to gpu 
    ids = batch['input_ids'].to(device, dtype = torch.long)
    mask = batch['attention_mask'].to(device, dtype = torch.long)
    labels = batch['labels'].to(device, dtype = torch.long)
    # returned value 
    logits = bertt(ids,mask)
    # loss
    loss = loss_fn(logits, labels)
    #total loss
    tr_loss += loss.item()
  
    nb_tr_steps += 1
    nb_tr_examples += labels.size(0)
    # loss at every 100 ite
    if idx % 100==0:
        loss_step = tr_loss/nb_tr_steps
        print(f"Training loss per 100 training steps: {loss_step}")

    flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
    active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

    active_accuracy = labels.view(-1) != -100

    labels = torch.masked_select(flattened_targets, active_accuracy)
    predictions = torch.masked_select(flattened_predictions, active_accuracy)
    tr_labels.extend(labels)
    tr_preds.extend(predictions)

    tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
    tr_accuracy += tmp_tr_accuracy
  
    torch.nn.utils.clip_grad_norm_(
        parameters=bertt.parameters(), max_norm=MAX_GRAD_NORM
    )
        
    optimizer.zero_grad()
    # back propagation
    loss.backward()
    optimizer.step()
  epoch_loss = tr_loss / nb_tr_steps
  tr_accuracy= tr_accuracy / nb_tr_steps
  print(f"Training loss epoch: {epoch_loss}")
  print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.485680341720581
Training loss per 100 training steps: 2.0068095267409145
Training loss per 100 training steps: 1.9640203231009679
Training loss per 100 training steps: 1.938724350889656
Training loss per 100 training steps: 1.9116133160127369
Training loss per 100 training steps: 1.8988819921801905
Training loss per 100 training steps: 1.8880609835245448
Training loss per 100 training steps: 1.8874738963796476
Training loss per 100 training steps: 1.8810114410337289
Training loss per 100 training steps: 1.8764276711578243
Training loss per 100 training steps: 1.8750850853029188
Training loss per 100 training steps: 1.8692599192843666
Training loss per 100 training steps: 1.8661638759355759
Training loss per 100 training steps: 1.8571280578665326
Training loss per 100 training steps: 1.8639614067700485
Training loss per 100 training steps: 1.8637437326442712
Training loss per 100 training steps: 1.8623453766982456
Training loss p

In [ ]:
# evaluating on validation data
def evaluate(bertt):
   
    # For each batch in our validation set...
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
  
    
    bertt.eval()
    # we do not need to train this time
    with torch.no_grad():
       for idx, batch in enumerate(validation_loader):
             ids = batch['input_ids'].to(device, dtype = torch.long)
             mask = batch['attention_mask'].to(device, dtype = torch.long)
             labels = batch['labels'].to(device, dtype = torch.long)
    
             logits = bertt(ids,mask)
             loss = loss_fn(logits, labels)
             tr_loss += loss.item()
  
             nb_tr_steps += 1
             nb_tr_examples += labels.size(0)
    
             if idx % 100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Training loss per 100 training steps: {loss_step}")

             flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
             active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
             flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

             active_accuracy = labels.view(-1) != -100

             labels = torch.masked_select(flattened_targets, active_accuracy)
             predictions = torch.masked_select(flattened_predictions, active_accuracy)
          
             tr_labels.extend(labels)
             tr_preds.extend(predictions)
            
             tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
             tr_accuracy += tmp_eval_accuracy
  
    labels = [ids_to_labels[id.item()] for id in tr_labels]
    predictions = [ids_to_labels[id.item()] for id in tr_preds]
    
    eval_loss = tr_loss / nb_tr_steps 
    eval_accuracy = tr_accuracy / nb_tr_steps 
    print(f"Validation Loss: { eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy }")

    return labels, predictions

In [ ]:
x = evaluate(bertt)

Training loss per 100 training steps: 1.2055777311325073
Training loss per 100 training steps: 1.7776960647932374
Training loss per 100 training steps: 1.7565792283015464
Training loss per 100 training steps: 1.7646943089177838
Training loss per 100 training steps: 1.7672031529229182
Training loss per 100 training steps: 1.8002258670663167
Training loss per 100 training steps: 1.7914915562370255
Training loss per 100 training steps: 1.7799972818258316
Training loss per 100 training steps: 1.7733305906051404
Training loss per 100 training steps: 1.7788940567056029
Training loss per 100 training steps: 1.7626267935221012
Training loss per 100 training steps: 1.7626738626641862
Training loss per 100 training steps: 1.7715490925297153
Training loss per 100 training steps: 1.7783049409500367
Training loss per 100 training steps: 1.7810151228876645
Training loss per 100 training steps: 1.7895806548418323
Training loss per 100 training steps: 1.7966949432194792
Training loss per 100 training 

In [ ]:
# saving model
PATH = "/content/drive/MyDrive/model.pth"

torch.save(bertt.state_dict(), PATH)

NameError: ignored

IsADirectoryError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


testing the model

In [ ]:
# importing test file
file_folder = r"hi_dev.conll"
file =  open(file_folder,encoding="utf-8")
text = file.read()


In [ ]:

s_cout = 0
l1 = []
l2 = []
l3 = []
for word in text.split('\n'):
  if(word == ''):
    s_cout =  s_cout + 1
  elif(word[0] == '#'):
    continue
    
  else:
      word = word.split(" ")
      k  = "sent_"+ str(s_cout)
      l1.append(k)
      l2.append(word[0])
      l3.append(word[-1])

In [ ]:
df.head()

NameError: ignored

In [ ]:
df = pd.DataFrame({"sent":l1,"word":l2,"tag":l3})
df.head()

,sent,word,tag
0,sent_0,१४९२,O
1,sent_0,में,O
2,sent_0,एक,O
3,sent_0,चार्टर,O
4,sent_0,के,O


In [ ]:
lis = []
for i in range(df.shape[0]):
  if(df.iloc[i,1] != ''):
    lis.append(i)
df = df.iloc[lis,:]

In [ ]:
lis = []
for i in range(df.shape[0]):
  if(df.iloc[i,2] != ''):
    lis.append(i)
df = df.iloc[lis,:]

In [ ]:
df['sentence'] =  df['word']
df['word_labels']  = df['tag']
df.head()

,sent,word,tag,sentence,word_labels
0,sent_0,१४९२,O,१४९२,O
1,sent_0,में,O,में,O
2,sent_0,एक,O,एक,O
3,sent_0,चार्टर,O,चार्टर,O
4,sent_0,के,O,के,O


In [ ]:
labels_to_ids = {k: v for v, k in enumerate(df.tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(df.tag.unique())}
labels_to_ids
ids_to_labels

{0: 'O',
 1: 'B-LOC',
 2: 'B-PROD',
 3: 'B-CORP',
 4: 'I-CORP',
 5: 'B-GRP',
 6: 'I-GRP',
 7: 'B-CW',
 8: 'B-PER',
 9: 'I-PER',
 10: 'I-CW',
 11: 'I-LOC',
 12: 'I-PROD'}

In [ ]:
df = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
df.head()

,sentence,word_labels
0,१४९२,O
1,में,O
2,एक,O
3,चार्टर,O
4,के,O


In [ ]:
df.shape

(3742, 2)

In [ ]:
testing_set = dataset(df, tokenizer, MAX_LEN)

In [ ]:
test_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# loading model from drive
bertt = clyas(True)
# auto model has pretrained bert parameters
PATH = "/content/drive/MyDrive/model.pth"
bertt.load_state_dict(torch.load(PATH))
bertt.to(device)


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
# evaluating model on the test file
def evaluate(bertt):
   
    # For each batch in our validation set...
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
  
    
    bertt.eval()
    with torch.no_grad():
       for idx, batch in enumerate(testing_loader):
             ids = batch['input_ids'].to(device, dtype = torch.long)
             mask = batch['attention_mask'].to(device, dtype = torch.long)
             labels = batch['labels'].to(device, dtype = torch.long)
    
             logits = bertt(ids,mask)
             loss = loss_fn(logits, labels)
             tr_loss += loss.item()
  
             nb_tr_steps += 1
             nb_tr_examples += labels.size(0)
    
             if idx % 100==0:
                loss_step = tr_loss/nb_tr_steps
                print(f"Training loss per 100 training steps: {loss_step}")

             flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
             active_logits = logits.view(-1,13) # shape (batch_size * seq_len, num_labels)
             flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len)

             active_accuracy = labels.view(-1) != -100

             labels = torch.masked_select(flattened_targets, active_accuracy)
             predictions = torch.masked_select(flattened_predictions, active_accuracy)
          
             tr_labels.extend(labels)
             tr_preds.extend(predictions)
            
             tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
             tr_accuracy += tmp_eval_accuracy
  
    labels = [ids_to_labels[id.item()] for id in tr_labels]
    predictions = [ids_to_labels[id.item()] for id in tr_preds]
    
    eval_loss = tr_loss / nb_tr_steps 
    eval_accuracy = tr_accuracy / nb_tr_steps 
    print(f"testing  Loss: { eval_loss}")
    print(f"testing Accuracy: {eval_accuracy }")

    return labels, predictions

In [ ]:
x = evaluate(bertt)

Training loss per 100 training steps: 1.6456727981567383
Training loss per 100 training steps: 1.5833555524892147
Training loss per 100 training steps: 1.5257385473939318
Training loss per 100 training steps: 1.5281290170362225
Training loss per 100 training steps: 1.5204096727834973
Training loss per 100 training steps: 1.5139267224513604
Validation Loss: 1.517461323632603
Validation Accuracy: 0.6733565107458912


In [ ]:
# .67 as f1 score
from sklearn.metrics import f1_score
f1_score(x[0],x[1], average='micro')

0.6733790661650808